In [9]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np


In [10]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Adj Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

tickers = ['AAPL', 'MSFT', 'GOOG', 'JNJ', 'XOM', 'TSLA', 'JPM', 'UNH', 'NVDA', 'PG']
start_date = '2019-01-01'
end_date = '2024-01-01'
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.tail())

[*********************100%%**********************]  10 of 10 completed

           Date        AAPL        MSFT        GOOG         JNJ         XOM  \
1253 2023-12-22  192.868149  142.370361  151.898438  164.497726  372.543945   
1254 2023-12-26  192.320206  142.470123  152.562851  165.470566  372.623505   
1255 2023-12-27  192.419830  141.093506  152.768051  166.463058  372.036713   
1256 2023-12-28  192.848206  140.933899  152.992767  167.347473  373.240112   
1257 2023-12-29  191.802170  140.584747  153.149124  167.150925  373.995972   

           TSLA         JPM         UNH        NVDA         PG  
1253  48.819527  142.584473  252.539993  516.077576  99.358162  
1254  49.268425  143.232239  256.609985  515.799866  99.582397  
1255  49.406395  143.350006  261.440002  518.537415  99.114418  
1256  49.511375  143.026123  253.179993  520.630249  97.681236  
1257  49.511375  143.821106  248.479996  522.187439  97.476486  


In [11]:
### Function to Compute Daily Returns
''' 
The percentage change in stock price from day t-1 to day t is given by:
    R(i,t) = [P(i,t) - P(i,t-1)]​ / P(i,t-1)
In our case, use the pre-built python pct_change() function
'''
def daily_returns(stock_data):
    # Drop the date column
    stock_data = stock_data.drop(columns = ['Date'])
    # Compute the percentage change
    rets = stock_data.pct_change().dropna()
    
    return rets

# Call the function to compute the daily returns for error checking
returns = daily_returns(stock_data)
print(returns.tail())

### Function to Compute Volatilities
def volatility(tickers, returns):
    # Create list to store volatilities
    volatilities = []  
    # Iterate over each ticker in tickers list
    for ticker in tickers:
        # Compute annualized volatility
        sigma = np.std(returns[ticker]) * np.sqrt(252)  
        volatilities.append(round(sigma, 5))  
        
    return volatilities

# Call the function to compute the volatility for each stock
vols = volatility(tickers, returns)
print(vols)


          AAPL      MSFT      GOOG       JNJ       XOM      TSLA       JPM  \
1253 -0.005547  0.006488  0.004004 -0.000597  0.002784 -0.003266  0.007071   
1254 -0.002841  0.000701  0.004374  0.005914  0.000214  0.009195  0.004543   
1255  0.000518 -0.009662  0.001345  0.005998 -0.001575  0.002800  0.000822   
1256  0.002226 -0.001131  0.001471  0.005313  0.003235  0.002125 -0.002259   
1257 -0.005424 -0.002477  0.001022 -0.001174  0.002025  0.000000  0.005558   

           UNH      NVDA        PG  
1253 -0.007701  0.000827  0.001769  
1254  0.016116 -0.000538  0.002257  
1255  0.018822  0.005307 -0.004699  
1256 -0.031594  0.004036 -0.014460  
1257 -0.018564  0.002991 -0.002096  
[0.32222, 0.31806, 0.19855, 0.31885, 0.30479, 0.51749, 0.21153, 0.64674, 0.29493, 0.34262]


In [12]:
### Function to build Risk-Parity Portfolio
''' 
The Risk-Parity Portfolio method allocates more weight to less volatile stocks and less weight to more volatile stocks, 
aiming to equalize the contribution of each stock to the total portfolio risk. Calculation is as follows:
    1. Estimate the volatility (standard deviation) of each stock's returns
    2. Assign weights inversely proportional to the volatility:
        w(i) = (1/σ(i)) / (sum_(j=1)^10 1/σ(j))
'''
def risk_parity_weights(volatilities):
    # Empty list to store weights
    weights = []
    # Inverse sum volatilities for denominator
    inv_sum_vols = sum(1 / vol for vol in volatilities)
    # Iterate over values in volatilities list
    for vol in volatilities:
        # Weights calculation
        weight = (1 / vol) / inv_sum_vols
        weights.append(round(weight, 5))

    return weights

weights = risk_parity_weights(vols)
print('The sum of the weights is:', round(sum(weights), 6), 'and should be 1')
print(weights)


The sum of the weights is: 0.99999 and should be 1
[0.09633, 0.09759, 0.15633, 0.09735, 0.10184, 0.05998, 0.14674, 0.04799, 0.10524, 0.0906]


In [13]:
### Compute Daily Portfolio Returns
''' 
The daily portfolio return is computed as the weighted sum of the returns of each stock for that particular day, equation:
   R_{portfolio,t} = (sum_(i=1)^10 w_i * R_{i,t})

For Each Day t (e.g., Day 1):
    Take the return of stock 1 on day 1 and multiply it by the weight of stock 1.
    Take the return of stock 2 on day 1 and multiply it by the weight of stock 2.
    Repeat this for all 10 stocks.

Then, sum all these weighted returns to get the portfolio return for day 1:
    R_{portfolio,t=1} = w_1R_{1,1} + w_2R_{2,1} +...+ w_10R_{10,1}
'''
# Compute the portfolio returns by multiplying the daily returns with the weights
portfolio_rets = returns.dot(weights)
# Convert portfolio rets to df
dates = stock_data['Date']
portfolio_rets_df = pd.DataFrame({
    'Date': dates,
    'Portfolio_Returns': portfolio_rets
})
# Display the portfolio returns
print(portfolio_rets_df.tail())


           Date  Portfolio_Returns
1253 2023-12-22           0.001670
1254 2023-12-26           0.003215
1255 2023-12-27           0.001065
1256 2023-12-28          -0.001425
1257 2023-12-29          -0.000463


In [14]:
### Build Variance-Covariance Matrix
''' 
The covariance between two stocks i and j is calculated as:
    Cov(R_i, R_j) = (1 / (T - 1)) * sum_{t=1}^T (R_{i,t} - R̄_i)(R_{j,t} - R̄_j)
Where:
    R_{i,t} and R_{j,t} are the daily returns of stocks i and j on day t,
    R̄_i and R̄_j are the average returns of stocks i and j over the period,
    T is the number of time periods
We will check our work using the pre-defined .cov() function
'''
def var_cov_matrix(tickers, daily_rets):
    # Initialize Empty Covariance Matrix
    num_stocks = len(daily_rets.columns)
    matrix = np.zeros((num_stocks, num_stocks))
    # Find T (number of days)
    T = len(returns)
    # Calculate the mean of each stock's returns
    mean_returns = daily_rets.mean()
    # Iterate to populate cov matrix
    for i in range(num_stocks):
        for j in range(num_stocks):
            # Covariance Matrix equation w/ indexing
            matrix[i, j] = (1 / (T - 1)) * np.sum((daily_rets.iloc[:, i] - mean_returns[i]) * (daily_rets.iloc[:, j] - mean_returns[j]))

    # Convert the covariance matrix to a DataFrame for readability
    cov_matrix_df = pd.DataFrame(matrix, index = daily_rets.columns, columns = daily_rets.columns)

    return cov_matrix_df

cov_matrix = var_cov_matrix(tickers, returns)
print(cov_matrix)


          AAPL      MSFT      GOOG       JNJ       XOM      TSLA       JPM  \
AAPL  0.000412  0.000274  0.000103  0.000186  0.000296  0.000435  0.000120   
MSFT  0.000274  0.000402  0.000086  0.000176  0.000290  0.000408  0.000100   
GOOG  0.000103  0.000086  0.000157  0.000104  0.000100  0.000095  0.000100   
JNJ   0.000186  0.000176  0.000104  0.000404  0.000174  0.000246  0.000105   
XOM   0.000296  0.000290  0.000100  0.000174  0.000369  0.000443  0.000124   
TSLA  0.000435  0.000408  0.000095  0.000246  0.000443  0.001064  0.000126   
JPM   0.000120  0.000100  0.000100  0.000105  0.000124  0.000126  0.000178   
UNH   0.000404  0.000333  0.000055  0.000226  0.000355  0.000666  0.000066   
NVDA  0.000171  0.000149  0.000121  0.000178  0.000173  0.000210  0.000117   
PG    0.000139  0.000130  0.000083  0.000256  0.000116  0.000169  0.000068   

           UNH      NVDA        PG  
AAPL  0.000404  0.000171  0.000139  
MSFT  0.000333  0.000149  0.000130  
GOOG  0.000055  0.000121  0.00

### Covariance Between Two Stocks:
Each element in the covariance matrix represents the covariance between two stocks. Covariance is a measure of how two stocks' returns move together:

- **Positive Covariance**: If the covariance between two stocks is positive, it means that their returns tend to move in the same direction. When one stock’s return increases, the other’s return is likely to increase as well.
- **Negative Covariance**: A negative covariance indicates that the returns of the two stocks move in opposite directions. When one stock’s return increases, the other’s return tends to decrease.
- **Zero Covariance**: If the covariance is close to zero, the two stocks are not strongly correlated, meaning their returns move independently of each other.

### Variance of Each Stock (Diagonal Elements):
The diagonal elements of the covariance matrix represent the variance of each individual stock, which is the covariance of the stock with itself.
Variance is a measure of the dispersion of a stock’s returns around its mean. A higher variance means the stock’s returns are more spread out, indicating higher volatility (risk).

For stock \(i\), the variance is:
    Var(R_i) = Cov(R_i, R_i)


In [19]:
### Calculate Portfolio Variance and Volatility
''' 
The portfolio variance is calculated as:
    Var(R_{portfolio}) = w^T \Sigma w
Where:
- w^T is the transpose of the weight vector
- Sigma is the covariance matrix
- w is the vector of portfolio weights
'''
def calc_var_and_vol(weights, cov_matrix):
    # Compute transpose of weights matrix
    weights_arr = np.array(weights) # Convert weights to an array
    transpose = weights_arr.T
    # Perform matrix multiplication (dot product) between covariance matrix and weights vector
    prod = np.dot(cov_matrix, weights)
    # Compute the dot product between prod and the weights transpose
    var = np.dot(transpose, prod)

    # Compute the portfolio volatility = sqrt(var)
    vol = np.sqrt(var)

    return var, vol

variance, volatility = calc_var_and_vol(weights, cov_matrix)
print('Variance:', variance)
print('Volatility:', volatility)


Variance: 0.0001822544689831416
Volatility: 0.013500165516879473


In [24]:
### Calculate Beta
''' 
The beta of a stock i is calculated as:
    Beta_i = Cov(R_i, R_m) / Var(R_m)
Where:
- R_i is the return of the stock
- R_m is the return of the market (we'll use the S&P 500)
- Cov(R_i, R_m) is the covariance between the stock and the market
- Var(R_m) is the variance of the market return
'''
def get_beta(daily_rets):
    # Create empty list to store betas
    betas = []
    # Get market returns
    mkt_data = yf.download('^GSPC', start = '2019-01-01', end = '2024-01-01')['Adj Close']  # S&P 500 index
    mkt_rets = mkt_data.pct_change().dropna()
    # Ensure daily_rets is indexed by the same dates as market_rets
    daily_rets = daily_rets.set_index(mkt_rets.index)
    
    # Iterate over each stock in daily_rets cols to calculate Cov(R_i, R_m)
    for stock in daily_rets.columns:
        # Calculate covariance between each stock and market
        cov_matrix = np.cov(daily_rets[stock], mkt_rets)
        # Covariance between the stock and market is at position [0, 1]
        covariance = cov_matrix[0, 1]
        # Calculate variance of market returns
        mkt_var = np.var(mkt_rets)
        # Calculate beta
        beta = covariance / mkt_var
        betas.append(round(beta, 5))

    return betas

# Function call to return Betas
betas = get_beta(returns)
print(betas)


[*********************100%%**********************]  1 of 1 completed

[1.21659, 1.1317, 0.52445, 1.09952, 1.18308, 1.73021, 0.58526, 1.50446, 0.86173, 0.88562]


In [ ]:
### Compute Expected Returns


In [ ]:
### Portfolio Expected Return
''' 
The expected return of the portfolio is calculated as the weighted sum of the individual stock returns:
    R_{portfolio} = \sum_{i=1}^{N} w_i*R_i
Where:
- R_i is the expected return of each stock
- w_i is the weight of each stock in the portfolio
'''
